# Twitter feed analysis using Twython

In [1]:
import pandas as pd
import json
from twython import Twython
# !pip3 install TwitterAPI --upgrade

# Search API

In [2]:
credentials = {}  
credentials['CONSUMER_KEY'] = 'xdud9FSorfoBo7uMAySUvZqwa' 
credentials['CONSUMER_SECRET'] = '6prpLIb5wgErEN1h9ilVrMCMLfwWoWBJDYcDmejjhqSrBQX7y3'   
credentials['ACCESS_TOKEN'] = '451935727-2CDFLGtGpDnVII1NWmRgzEJ1PlngviCtmKrYpwpi'  
credentials['ACCESS_SECRET'] = 'tKcHhnyNrYKpblkqOyZPOdy4YKpNpQ5D88VrTDpMOKHpk'

In [3]:
# Save the credentials object to file
with open("twitter_credentials.json", "w") as file:  
    json.dump(credentials, file)

In [4]:
# Load credentials from json file
with open("twitter_credentials.json", "r") as file:  
    creds = json.load(file)

In [5]:
# !pip install twython

In [6]:
# Instantiate an object
python_tweets = Twython(creds['CONSUMER_KEY'], creds['CONSUMER_SECRET'])

In [7]:
query = {'q': 'India',  
        'result_type': 'popular',
        'count': 10,
        'lang': 'en',
        }

In [8]:
dict_ = {'user': [], 'date': [], 'text': [], 'favorite_count': []}
for status in python_tweets.search(**query)['statuses']:  
    dict_['user'].append(status['user']['screen_name'])
    dict_['date'].append(status['created_at'])
    dict_['text'].append(status['text'])
    dict_['favorite_count'].append(status['favorite_count'])

In [9]:
df = pd.DataFrame(dict_)

In [10]:
df.sort_values(by='date', inplace=True, ascending=False)  
df.head(10)  

,user,date,text,favorite_count
0,narendramodi,Wed Mar 13 12:40:22 +0000 2019,No place for middlemen.\n\nNo tolerance for co...,30857
4,narendramodi,Wed Mar 13 05:30:30 +0000 2019,"My fellow Indians, \n\nUrging you all to stren...",57366
8,narendramodi,Wed Mar 13 05:12:14 +0000 2019,"Dear @msdhoni, @imVkohli and @ImRo45, \nYou ar...",36062
1,narendramodi,Wed Mar 13 05:01:03 +0000 2019,"Respected @CitiznMukherjee, \nBeing among Indi...",22154
2,narendramodi,Wed Mar 13 04:42:38 +0000 2019,"Dear @RNTata2000, @anandmahindra and @ashishch...",20457
5,narendramodi,Wed Mar 13 04:39:37 +0000 2019,"India is proud of @Neeraj_chopra1, @DuttYogi a...",13944
7,narendramodi,Wed Mar 13 04:15:26 +0000 2019,Four requests for a stronger democracy.\n\nAn ...,17331
6,AskAnshul,Tue Mar 12 12:11:03 +0000 2019,"Whenever Israel faces terror attack, Western M...",10991
3,pewdiepie,Tue Mar 12 11:04:21 +0000 2019,tseries now have people spamming my diss track...,117253
9,AJEnglish,Tue Mar 12 10:00:00 +0000 2019,Cow vigilantism is on the rise in India. https...,5835


# Streaming API

In [11]:
from twython import TwythonStreamer  
import csv

In [12]:
def process_tweet(tweet):  
    d = {}
    d['hashtags'] = [hashtag['text'] for hashtag in tweet['entities']['hashtags']]
    d['text'] = tweet['text']
    d['user'] = tweet['user']['screen_name']
    d['user_loc'] = tweet['user']['location']
    return d

In [13]:
class MyStreamer(TwythonStreamer):
    
    # Received data
    def on_success(self, data):
        
    # Only collect tweets in English
        if data['lang'] == 'en':
            tweet_data = process_tweet(data)
            self.save_to_csv(tweet_data)
    
    # Problem with the API
    def on_error(self, status_code, data):
        print(status_code, data)
        self.disconnect()
    
    # Save each tweet to csv file
    def save_to_csv(self, tweet):
        with open(r'saved_tweets.csv', 'a', encoding='utf-8') as file:
            writer = csv.writer(file)
            writer.writerow(list(tweet.values()))

In [14]:
# Instantiate from our streaming class
stream = MyStreamer(creds['CONSUMER_KEY'], creds['CONSUMER_SECRET'],  
                    creds['ACCESS_TOKEN'], creds['ACCESS_SECRET'])

In [18]:
# Start the stream
# stream.statuses.filter(track = 'peace')

In [19]:
import ast
from collections import Counter

In [20]:
tweets = pd.read_csv("saved_tweets.csv", header = None, names = ['hashtags','text','user','location'])

In [21]:
tweets.head()

,hashtags,text,user,location
0,[],@op_hawkeye @NSAGov @HonoredValor @SEALofHonor...,andydeerman118,"Queens, NY"
1,[],RT @SaharaReporters: EXCLUSIVE: Aregbesola Hol...,tomjerryng,NaN
2,[],If you start a chain reaction for peace on thi...,ashscheherazade,"Washington, DC"
3,[],RT @QaisiSaleem: @BBCArabic Please ..Save the ...,OmMarya53799397,NaN
4,[],RT @ShahNafisa: Peace overtures r welcome but ...,MirAsifKhanDas2,NaN


In [23]:
list_hashtag_strings = [entry for entry in tweets.hashtags]
list_hashtag_lists = ast.literal_eval(','.join(list_hashtag_strings))
hashtag_list = [ht.lower() for list_ in list_hashtag_lists for ht in list_ ]

In [24]:
counter_hashtags = Counter(hashtag_list)  
counter_hashtags.most_common(15)

[('prayforlagosisland', 6),
 ('peace', 5),
 ('youknewthat', 3),
 ('adn191stweeksary', 3),
 ('travel', 3),
 ('nature', 3),
 ('photo', 3),
 ('home', 3),
 ('relax', 3),
 ('forest', 3),
 ('country', 3),
 ('otd', 3),
 ('thequeenmzansi', 3),
 ('bethegeneration', 2),
 ('stopcyberwarfare', 2)]

In [25]:
from geopy.geocoders import Nominatim  
import gmplot
# ! pip install geopy
# ! pip install gmplot

In [26]:
geolocator = Nominatim(user_agent="graduate student self learning")

# Go through all tweets and add locations to 'coordinates' dictionary
coordinates = {'latitude': [], 'longitude': []}  
for count, user_loc in enumerate(tweets.location):  
    try:
        location = geolocator.geocode(user_loc)

        # If coordinates are found for location
        if location:
            coordinates['latitude'].append(location.latitude)
            coordinates['longitude'].append(location.longitude)

    # If too many connection requests
    except:
        pass

# Instantiate and center a GoogleMapPlotter object to show our map
gmap = gmplot.GoogleMapPlotter(30, 0, 3)

# Insert points on the map passing a list of latitudes and longitudes
gmap.heatmap(coordinates['latitude'], coordinates['longitude'], radius=20)

# Save the map to html file
gmap.draw("python_heatmap.html")  